In [1]:
%run mainTNR.py

RGstep:  0  , Truncation Errors: [2.008217989894428e-06, 4.710099382075902e-05, 4.3385250754765434e-05, 0.00010246157077446297]
time spent for this RGstep:  0.8481032848358154
shape of A: (6, 6, 6, 6)
shape of q: (4, 4, 6)
shape of s: (6, 4)
shape of u: (4, 4, 4, 4)
shape of y: (4, 4, 4)
shape of v: (4, 4, 6)
shape of w: (4, 4, 6)
RGstep:  1  , Truncation Errors: [1.2052261389767233e-06, 5.5314341863221894e-05, 8.854347959034827e-05, 0.00011995394719820866]
time spent for this RGstep:  0.8429150581359863
shape of A: (6, 6, 6, 6)
shape of q: (6, 6, 6)
shape of s: (6, 4)
shape of u: (6, 6, 4, 4)
shape of y: (6, 4, 4)
shape of v: (6, 6, 6)
shape of w: (4, 4, 6)
RGstep:  2  , Truncation Errors: [1.3926494470403928e-06, 6.887552980815456e-05, 8.107328747758827e-05, 0.00010534040724117942]
time spent for this RGstep:  0.7530243396759033
shape of A: (6, 6, 6, 6)
shape of q: (6, 6, 6)
shape of s: (6, 4)
shape of u: (6, 6, 4, 4)
shape of y: (6, 4, 4)
shape of v: (6, 6, 6)
shape of w: (4, 4, 6)


In [1]:
%run mainScInvTNR.py

Iteration:  0  of  2000 , Trunc. Error: 6.08743e-10,4.06838e-06
Iteration:  100  of  2000 , Trunc. Error: 6.08743e-10,2.94729e-06
Iteration:  200  of  2000 , Trunc. Error: 6.08743e-10,2.48122e-06
Iteration:  300  of  2000 , Trunc. Error: 6.08743e-10,2.1954e-06
Iteration:  400  of  2000 , Trunc. Error: 6.08743e-10,1.98306e-06
Iteration:  500  of  2000 , Trunc. Error: 6.08743e-10,1.81134e-06
Iteration:  600  of  2000 , Trunc. Error: 6.08743e-10,1.67127e-06
Iteration:  700  of  2000 , Trunc. Error: 6.08743e-10,1.55939e-06
Iteration:  800  of  2000 , Trunc. Error: 6.08743e-10,1.4727e-06
Iteration:  900  of  2000 , Trunc. Error: 6.08743e-10,1.40773e-06
Iteration:  1000  of  2000 , Trunc. Error: 6.08743e-10,1.36077e-06
Iteration:  1100  of  2000 , Trunc. Error: 6.08743e-10,1.32807e-06
Iteration:  1200  of  2000 , Trunc. Error: 6.08743e-10,1.30612e-06
Iteration:  1300  of  2000 , Trunc. Error: 6.08743e-10,1.29188e-06
Iteration:  1400  of  2000 , Trunc. Error: 6.08743e-10,1.2829e-06
RGstep: 0 

In [3]:
print("Adiff: ",Adiff)
sclev = np.argmin(Adiff)
print("sclev: ",sclev)

Adiff:  [1, 0.034990588653356657, 0.002765012685538419, 0.0008438292308551482, 0.0017295999703319208, 0.002633914807284678, 0.004900118624230868, 0.009765734271955366, 0.019587028465820264, 0.03923223327970642]
sclev:  3


In [6]:
np.save('ScInvTensors',(A[sclev],qC[sclev],sC[sclev],yC[sclev],vC[sclev],wC[sclev]))

In [41]:
import numpy as np
chiK = 20
(Ax,qx,sx,yx,vx,wx)=np.load('ScInvTensors.npy')

In [43]:
from doScInvTNR import eigCut

##### Compress vertical indices
chitemp = Ax.shape[3]
genv = np.einsum(Ax,[1,2,5,21],Ax,[3,4,5,22],Ax,[1,2,6,23],Ax,[3,4,6,24],order='C').reshape(chitemp**2,chitemp**2)
dtemp, gtemp = eigCut(genv, chimax = chiK, dtol = 0)
gx = gtemp.reshape(chitemp,chitemp,gtemp.shape[1])

##### Compress horizontal indices
chitemp = Ax.shape[2]
renv = np.einsum(Ax,[1,2,22,5],Ax,[4,3,21,5],Ax,[1,2,24,6],Ax,[4,3,23,6],order='C').reshape(chitemp**2,chitemp**2)
dtemp, rtemp = eigCut(renv, chimax = chiK, dtol = 0)
rx = rtemp.reshape(chitemp,chitemp,rtemp.shape[1])

##### Compute gates
Aqs = np.einsum(Ax,[22,21,1,2],qx,[1,2,3],sx,[3,23],order='C')

rT = np.einsum(rx,[10,12,22],vx,[5,6,12],Aqs,[4,5,24],Aqs,[4,6,11],rx,[3,2,21],
               yx,[2,1,11],yx,[3,1,13],vx,[9,8,10],Aqs,[7,8,13],Aqs,[7,9,23],order='C')

gT = np.einsum(gx,[12,11,22],wx,[6,5,12],yx,[4,5,23],yx,[4,6,10],
    gx,[2,3,21],Aqs,[2,1,10],Aqs,[3,1,13],wx,[8,9,11],yx,[7,8,13],yx,[7,9,24],order='C')

In [44]:
np.save('rTgT',(rT,gT))

In [2]:
import numpy as np
(rT,gT)=np.load('rTgT.npy')

In [47]:
import h5py
with h5py.File("rT.h5") as h5f2:
    h5f2.create_dataset('rT', data=rT)
with h5py.File("gT.h5") as h5f:
    h5f.create_dataset('gT', data=gT)